# mere copywork of astonishing 7th place solution to the home credit competition
[original work](https://www.kaggle.com/jsaguiar/lightgbm-7th-place-solution)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import gc
import time
import numpy as np
import pandas as pd
from contextlib import contextmanager
import multiprocessing as mp
from functools import partial
from scipy.stats import kurtosis, iqr, skew
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
debug = True

In [ ]:
num_rows = 30000 if debug else None

In [ ]:
# df = get_train_test(DATA_DIRECTORY, num_rows=num_rows)

In [ ]:
NUM_THREADS = 4
DATA_DIRECTORY = '../input/home-credit-default-risk/'
SUBMISSION_SUFIX = "_model2_04"

In [ ]:
path = DATA_DIRECTORY
num_rows = num_rows

In [ ]:
train = pd.read_csv(os.path.join(path, 'application_train.csv'), nrows= num_rows)
test = pd.read_csv(os.path.join(path, 'application_test.csv'), nrows= num_rows)

In [ ]:
train.head()

In [ ]:
train['TARGET'].value_counts().plot.bar()

imbalanced data

In [ ]:
df = train.append(test)

In [ ]:
del train, test;

In [ ]:
gc.collect()

# preprocessing

## EDA는 각자 공부 혼자 할 수 있어야.

In [ ]:
df['CODE_GENDER'].value_counts()

In [ ]:
df['CODE_GENDER'] != 'XNA'

In [ ]:
df = df[df['CODE_GENDER'] != 'XNA'] # 4 people with XNA code gender

In [ ]:
df = df[df['AMT_INCOME_TOTAL'] < 20000000]  # Max income in test is 4M; train has a 117M value

In [ ]:
(df['DAYS_EMPLOYED'] ==365243).sum()

In [ ]:
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

In [ ]:
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

In [ ]:
[col for col in df.columns if 'FLAG_DOC' in col]

In [ ]:
docs = [col for col in df.columns if 'FLAG_DOC' in col]

In [ ]:
df[docs].sum(axis=1)

In [ ]:
df['DOCUMENT_COUNT'] = df[docs].sum(axis=1)

In [ ]:
len(docs)

In [ ]:
df['DOCUMENT_COUNT'].hist()

In [ ]:
df[docs].kurtosis(axis=1).hist()

In [ ]:
df['NEW_DOC_KURT'] = df[docs].kurtosis(axis=1)

In [ ]:
- df['DAYS_BIRTH'] / 365 # 나이 계산

In [ ]:
def get_age_label(days_birth):
    """ Return the age group label (int). """
    age_years = -days_birth / 365
    if age_years < 27: return 1
    elif age_years < 40: return 2
    elif age_years < 50: return 3
    elif age_years < 65: return 4
    elif age_years < 99: return 5
    else: return 0

In [ ]:
df['AGE_RANGE'] = df['DAYS_BIRTH'].apply(lambda x : get_age_label(x))

In [ ]:
df['EXT_SOURCE_1'] * df['EXT_SOURCE_2']

In [ ]:
df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']

+ EXT_SOURCE_1 이 feature importance가 높게 나오는 변수였는데 뭔지는 유추하지 못했었다고 함.
+ 그래도 중요도가 높게 나오니까 중요한 것 * 중요한 것으로 더 좋은 변수를 만들어보고자 하는 시도

In [ ]:
df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']

In [ ]:
# df['EXT_SOURCES_PROD_12'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2']

In [ ]:
# df['EXT_SOURCES_PROD_23'] = df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']

+ 현업에서는 설명 가능한 피처를 쓰지만 캐글에서는 설명이 되지 않아도 랭킹 올릴 수 있다면 쓴다고 한다.

In [ ]:
df['EXT_SOURCES_WEIGHTED'] = df['EXT_SOURCE_1'] * 2 + df['EXT_SOURCE_2'] * 1 + df['EXT_SOURCE_3'] * 3

+ 다른 사람들이 시험해 봤는데 이렇게 변경해서 사용하는 것이 성능이 좋았다고 함.
+ 혼자 푸는 캐글이 아니다.

In [ ]:
np.warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')
# 이건 왜하지?

In [ ]:
np.min([500, 600])

In [ ]:
eval('np.min([500, 600])')

In [ ]:
for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
        feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
        df[feature_name] = eval('np.{}'.format(function_name))(
            df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

# credit ratio

In [ ]:
df['CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
# annuity 연금 - 상식적으로 재정상태를 볼 수 있는 변수

In [ ]:
df['CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
# 구매한 상품 가격 대비 빚

In [ ]:
df['ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
# 수입 대비 연금

In [ ]:
df['CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
# 수입 대비 빚

In [ ]:
df['INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
# 고용기간 대비 수입

In [ ]:
df['INCOME_TO_BIRTH_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_BIRTH']
# 나이 대비 수입

In [ ]:
df['EMPLOYED_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
# 나이 대비 일한 기간

In [ ]:
df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']
# 나이 대비 회사에 id 만들어진 지 얼마나 되었는가

In [ ]:
df['CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']


In [ ]:
df['CAR_TO_EMPLOYED_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']

In [ ]:
df['PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']

In [ ]:
# 중요한 groupby
# statistics for applications in the same group
group = ['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'AGE_RANGE', 'CODE_GENDER']

In [ ]:
# 그룹을 나누고 그룹 별 통계량을 피쳐로 쓰고 싶은 것

def do_median(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].median().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df


In [ ]:
df, group_cols, counted, agg_name = df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_MEDIAN'

In [ ]:
group_cols

In [ ]:
counted

In [ ]:
agg_name

In [ ]:
group_cols + [counted]

In [ ]:
df[group_cols]

In [ ]:
df[group_cols + [counted]]

In [ ]:
df[group_cols + [counted]].groupby(group_cols)

In [ ]:
df[group_cols + [counted]].groupby(group_cols)[counted]

In [ ]:
df[group_cols + [counted]].groupby(group_cols)[counted].median().reset_index()

In [ ]:
df[group_cols + [counted]].groupby(group_cols)[counted].median().reset_index().rename(columns={counted:agg_name})

In [ ]:
gp = df[group_cols + [counted]].groupby(group_cols)[counted].median().reset_index().rename(columns={counted:agg_name})

In [ ]:
gp.head()

In [ ]:
# merge 
df.merge(gp, on=group_cols, how='left')

In [ ]:
agg = {}
agg['counted'] = ['max', 'min', 'median', 'std', 'mean']

In [ ]:
agg

In [ ]:
# df[group_cols + [counted]].groupby(group_cols)[counted].agg(agg)

+ 상기 에러는 pandas aggregation 메소드의 deprecated 용법에 따른 것.   
+ [참고: 1st deprecation 참조](https://pandas.pydata.org/pandas-docs/stable/whatsnew/v0.20.0.html#whatsnew-0200-api-breaking-deprecate-group-agg-dict)

In [ ]:
df[group_cols + [counted]].groupby(group_cols)[counted].agg(agg['counted'])

In [ ]:
df[group_cols + [counted]].groupby(group_cols)[counted].agg(agg['counted']).reset_index()

In [ ]:
FOO = df[group_cols + [counted]].groupby(group_cols)[counted].agg(agg['counted']).reset_index()

In [ ]:
FOO.columns

In [ ]:
for i in FOO.columns:
    print(i)

In [ ]:
for i in FOO.columns.values:
    print(i)

In [ ]:
'_'.join('median')

In [ ]:
['_'.join(col) for col in FOO.columns[-5:]]

In [ ]:
['_'.join(col) for col in FOO.columns[-5:]]

In [ ]:
['counted_' + col for col in FOO.columns[-5:]]

In [ ]:
FOO.columns = ['_'.join(col).strip() for col in FOO.columns.values]

In [ ]:
FOO 

+ `agg`로 형성되는 데이터 프레임 형태가 다르기 때문에 string join으로 형성되는 이름도 달라진다. 

In [ ]:
del FOO

In [ ]:
def do_mean(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].mean().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df


def do_median(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].median().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df


def do_std(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].std().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df


def do_sum(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df


In [ ]:
group = ['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'AGE_RANGE', 'CODE_GENDER']
df = do_median(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_MEDIAN')
df = do_std(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_STD')
df = do_mean(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_MEAN')
df = do_std(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_STD')
df = do_mean(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_MEAN')
df = do_std(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_STD')
df = do_mean(df, group, 'AMT_CREDIT', 'GROUP_CREDIT_MEAN')
df = do_mean(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_MEAN')
df = do_std(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_STD')

In [ ]:
def label_encoder(df, categorical_columns=None):
    """Encode categorical values as integers (0,1,2,3...) with pandas.factorize. """
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    for col in categorical_columns:
        df[col], uniques = pd.factorize(df[col])
    return df, categorical_columns

In [ ]:
df = df
categorical_columns=None

In [ ]:
if not categorical_columns:
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']

In [ ]:
categorical_columns

In [ ]:
# sklearn label_encoder, pd factorizer 두가지 방법
for col in categorical_columns:
    break

In [ ]:
pd.factorize(df[col])

In [ ]:
for col in categorical_columns:
    df[col], uniques = pd.factorize(df[col])

In [ ]:
df[col]

In [ ]:
uniques

In [ ]:
def label_encoder(df, categorical_columns=None):
    """Encode categorical values as integers (0,1,2,3...) with pandas.factorize. """
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    for col in categorical_columns:
        df[col], uniques = pd.factorize(df[col])
    return df, categorical_columns

In [ ]:
df, le_encoded_cols = label_encoder(df, None)

In [ ]:
df[group_cols[0]]

In [ ]:
def drop_application_columns(df):
    """ Drop features based on permutation feature importance. """
    drop_list = [
        'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START',
        'FLAG_EMP_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL', 'FLAG_PHONE',
        'FLAG_OWN_REALTY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
        'REG_CITY_NOT_WORK_CITY', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR', 
        'COMMONAREA_MODE', 'NONLIVINGAREA_MODE', 'ELEVATORS_MODE', 'NONLIVINGAREA_AVG',
        'FLOORSMIN_MEDI', 'LANDAREA_MODE', 'NONLIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MODE',
        'FLOORSMIN_AVG', 'LANDAREA_AVG', 'FLOORSMIN_MODE', 'LANDAREA_MEDI',
        'COMMONAREA_MEDI', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'BASEMENTAREA_AVG',
        'BASEMENTAREA_MODE', 'NONLIVINGAPARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 
        'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG', 'YEARS_BUILD_MEDI', 'ENTRANCES_MODE',
        'NONLIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'LIVINGAPARTMENTS_MEDI',
        'YEARS_BUILD_MODE', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_MEDI', 'LIVINGAREA_MEDI',
        'YEARS_BEGINEXPLUATATION_MODE', 'NONLIVINGAPARTMENTS_AVG', 'HOUSETYPE_MODE',
        'FONDKAPREMONT_MODE', 'EMERGENCYSTATE_MODE'
    ]
    # Drop most flag document columns
    for doc_num in [2,4,5,6,7,9,10,11,12,13,14,15,16,17,19,20,21]:
        drop_list.append('FLAG_DOCUMENT_{}'.format(doc_num))
    df.drop(drop_list, axis=1, inplace=True)
    return df

# 왜 드랍을 해야 하는가?
# eda 해서 결정 한 것
[null importance](https://www.kaggle.com/ogrellier/feature-selection-with-null-importances)   
[permutation importance](https://academic.oup.com/bioinformatics/article/26/10/1340/193348)
+ 목적이 대출 상환 예측
+ 학습 하니까 feature 1이 중요
    + 일부러 타겟을 꼬아주고나서 학습을 시킴 feature1이 더이상 중요하지 않다 -> feature1은 타겟과 상관관계가 크다
+ 학습하니까 feature2가 중요 x
    + 타겟 꼬아서 학습시키니 feature2가 중요해졌다. feature2 중요 X
+ 개인적으로 공부하는 것이 좋은 노트북입니다.
  + feature를 많이 버렸다.

In [ ]:
def drop_application_columns(df):
    """ Drop features based on permutation feature importance. """
    drop_list = [
        'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START',
        'FLAG_EMP_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL', 'FLAG_PHONE',
        'FLAG_OWN_REALTY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
        'REG_CITY_NOT_WORK_CITY', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR', 
        'COMMONAREA_MODE', 'NONLIVINGAREA_MODE', 'ELEVATORS_MODE', 'NONLIVINGAREA_AVG',
        'FLOORSMIN_MEDI', 'LANDAREA_MODE', 'NONLIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MODE',
        'FLOORSMIN_AVG', 'LANDAREA_AVG', 'FLOORSMIN_MODE', 'LANDAREA_MEDI',
        'COMMONAREA_MEDI', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'BASEMENTAREA_AVG',
        'BASEMENTAREA_MODE', 'NONLIVINGAPARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 
        'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG', 'YEARS_BUILD_MEDI', 'ENTRANCES_MODE',
        'NONLIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'LIVINGAPARTMENTS_MEDI',
        'YEARS_BUILD_MODE', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_MEDI', 'LIVINGAREA_MEDI',
        'YEARS_BEGINEXPLUATATION_MODE', 'NONLIVINGAPARTMENTS_AVG', 'HOUSETYPE_MODE',
        'FONDKAPREMONT_MODE', 'EMERGENCYSTATE_MODE'
    ]
    # Drop most flag document columns
    for doc_num in [2,4,5,6,7,9,10,11,12,13,14,15,16,17,19,20,21]:
        drop_list.append('FLAG_DOCUMENT_{}'.format(doc_num))
    df.drop(drop_list, axis=1, inplace=True)
    return df


In [ ]:
df = drop_application_columns(df)

In [ ]:
# bureau_df = get_bureau(DATA_DIRECTORY, num_rows= num_rows)

In [ ]:
# 외부 데이터 가져오는 것. 다른 회사의 신용 평가 데이터
bureau = pd.read_csv(os.path.join(path, 'bureau.csv'), nrows= num_rows)

In [ ]:
bureau

In [ ]:
#credit 남은 정도 : -는 끝난것 +는 남은것
bureau['DAYS_CREDIT_ENDDATE']

In [ ]:
bureau['DAYS_CREDIT_ENDDATE'].max()

In [ ]:
bureau['DAYS_CREDIT_ENDDATE'].min()

In [ ]:
# Credit duration and credit/account end date difference
bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
# Credit to debt ratio and difference
bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']

In [ ]:
def one_hot_encoder(df, categorical_columns=None, nan_as_category=True):
    """Create a new column for each categorical value in categorical columns. """
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    categorical_columns = [c for c in df.columns if c not in original_columns]
    return df, categorical_columns

In [ ]:
# bureau, categorical_cols = one_hot_encoder(bureau, nan_as_category= False)
# null value를 카테고리로 치지 않고 null data로 유지
# light gbm은 null data 무시하고 학습하고 null data 채워주지 않아도 됨.

In [ ]:
bureau, categorical_cols = one_hot_encoder(bureau, nan_as_category= False)

In [ ]:
def get_bureau_balance(path, num_rows= None):
    bb = pd.read_csv(os.path.join(path, 'bureau_balance.csv'), nrows= num_rows)
    bb, categorical_cols = one_hot_encoder(bb, nan_as_category= False)
    # Calculate rate for each category with decay
    bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()
    # Min, Max, Count and mean duration of payments (months)
    agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}
    bb_processed = group_and_merge(bb, bb_processed, '', agg, 'SK_ID_BUREAU')
    del bb; gc.collect()
    return bb_processed


In [ ]:
bb = pd.read_csv(os.path.join(path, 'bureau_balance.csv'), nrows= num_rows)

In [ ]:
bb.head()

In [ ]:
bb['STATUS'].head()

+ 대출회사의 loan의 형ㅌㅐ

In [ ]:
bb, categorical_cols = one_hot_encoder(bb, nan_as_category= False)

In [ ]:
bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()

In [ ]:
bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()

In [ ]:
agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}

In [ ]:
# bb_processed = group_and_merge(bb, bb_processed, '', agg, 'SK_ID_BUREAU')

In [ ]:
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)

In [ ]:
df_to_agg = bb
df_to_merge = bb_processed
prefix = ''
aggregations = agg
aggregate_by = 'SK_ID_BUREAU'

In [ ]:
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

In [ ]:
df_to_agg

In [ ]:
aggregate_by = 'SK_ID_BUREAU'
aggregations = agg
df_to_agg.groupby(aggregate_by).agg(aggregations)

In [ ]:
agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)

In [ ]:
agg_df.columns

In [ ]:
pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])

In [ ]:
agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])

In [ ]:
agg_df.head()

In [ ]:
agg_df.reset_index()

In [ ]:
agg_df.head()

In [ ]:
agg_df = agg_df.reset_index()

In [ ]:
agg_df.head()

In [ ]:
'''
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)
'''

In [ ]:
df_to_merge.merge(agg_df, how='left', on=aggregate_by)

In [ ]:
'''
def get_bureau_balance(path, num_rows= None):
    bb = pd.read_csv(os.path.join(path, 'bureau_balance.csv'), nrows= num_rows)
    bb, categorical_cols = one_hot_encoder(bb, nan_as_category= False)
    # Calculate rate for each category with decay
    bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()
    # Min, Max, Count and mean duration of payments (months)
    agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}
    bb_processed = group_and_merge(bb, bb_processed, '', agg, 'SK_ID_BUREAU')
    del bb; gc.collect()
    return bb_processed
'''

In [ ]:
bureau = bureau.merge(get_bureau_balance(path, num_rows), how='left', on='SK_ID_BUREAU')

In [ ]:
bureau['STATUS_5']

In [ ]:
bureau['STATUS_12345'] = 0

In [ ]:
for i in range(1, 6):
    bureau['STATUS_12345'] += bureau['STATUS_{}'.format(i)]

In [ ]:
bureau['STATUS_12345'].unique()

In [ ]:
bureau['STATUS_12345'].value_counts()

+ aggregate by number of months in balance and merge with bureau(loan length agg)

In [ ]:
features = ['AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'DEBT_PERCENTAGE', 'DEBT_CREDIT_DIFF', 'STATUS_0', 'STATUS_12345']

In [ ]:
features

In [ ]:
# month_balance_size로 그룹을 만들고 numerical feature들의 평균 보기
bureau.groupby('MONTHS_BALANCE_SIZE')[features].mean()
#이게 또 하나의 정보가 될 수 있다.

In [ ]:
agg_length = bureau.groupby('MONTHS_BALANCE_SIZE')[features].mean().reset_index()

In [ ]:
# agg_length.rename({feat: 'LL_' + feat for feat in features}, axis=1, inplace=True)

In [ ]:
{feat: 'LL_' + feat for feat in features}

In [ ]:
agg_length.rename({feat: 'LL_' + feat for feat in features}, axis=1, inplace=True)

In [ ]:
agg_length.head()

In [ ]:
bureau.merge(agg_length, how='left', on='MONTHS_BALANCE_SIZE')

In [ ]:
bureau = bureau.merge(agg_length, how='left', on='MONTHS_BALANCE_SIZE')

## 2강 끝
---
# 3강